# 혼공머 05-2
> 교차 검증과 그리드 서치

- toc:true
- branch: master
- badges: true
- comments: true
- author: 김동준
- categories : ["Python", "혼공머"]

# 주로쓰는 패키지들

In [42]:
import numpy as np #넘파이
import pandas as pd #판다스
from plotnine import *  #플롯나인
import matplotlib.pyplot as plt #맷플랏립
import plotly.express as px #플랏리 상호작용 그래프
from IPython.display import HTML #블로그에 html로 올리려고 변환하는 패키지
import seaborn as sns # 씨본, 히스토그램 깔끔하게 그리는 패키지
#___________________________________
from sklearn.neighbors import KNeighborsClassifier # k 최근접이웃
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor  # 결정계수 
from sklearn.metrics import mean_absolute_error # 타깃과 예측의 절댓값 오차 평균을 반환
from sklearn.linear_model import LinearRegression # 선형 회귀
from sklearn.preprocessing import PolynomialFeatures #다중회귀로의 변환기
from sklearn.preprocessing import StandardScaler #규제
from sklearn.linear_model import Ridge #릿지
from sklearn.linear_model import Lasso #라쏘
from sklearn.linear_model import LogisticRegression # 로지스틱회귀
from scipy.special import expit #시그모이드함수
from scipy.special import softmax #소프트맥스함수
from sklearn.linear_model import SGDClassifier # 확률적 경사 하강법
from sklearn.tree import DecisionTreeClassifier # 트리
from sklearn.tree import plot_tree # 트리 모형
from sklearn.model_selection import cross_validate # 교차 검증
from sklearn.model_selection import StratifiedKFold # Kfold 교차 검증
from sklearn.model_selection import GridSearchCV # 그리드 서치 (하이퍼 파라미터 튜닝)
from scipy.stats import uniform, randint #랜덤 서치
from sklearn.model_selection import RandomizedSearchCV # 랜덤 서치 클래스

_________

# 검증 세트
> 테스트 세트를 사용하지 않고 이를 측정하기위해 훈련 세트를 또 나누는 것

> 보통 20~30% 를 테스트 세트와 검증 세트로 떼어 놓음

In [3]:
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [4]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [5]:
train_input , test_input , train_target , test_target = \
train_test_split(data, target , test_size=0.2 , random_state=42)

In [6]:
# 검증 세트 만들기
sub_input , val_input , sub_target , val_target = \
train_test_split(train_input, train_target , test_size=0.2 , random_state=42)

In [8]:
print(sub_input.shape , val_input.shape)

(4157, 3) (1040, 3)


In [9]:
# 모델 평가
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input , sub_target)
print(dt.score(sub_input ,sub_target))
print(dt.score(val_input ,val_target))

0.9971133028626413
0.864423076923077


> 훈련 세트에 과대적합

# 교차 검증 (임포트 필요)

> 검증 세트가 불안정한 것을 방지

> 검증 세트를 떼어내어 평가하는 과정을 여러 번 반복

> 교차 검증을 할 때 훈련 세트를 섞으려면 분할기를 지정해야한다.

In [11]:
scores = cross_validate(dt , train_input , train_target)
print(scores)

{'fit_time': array([0.00535083, 0.00490546, 0.00474167, 0.00510621, 0.00448918]), 'score_time': array([0.00066662, 0.00050211, 0.00046086, 0.00050902, 0.0004313 ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [12]:
print(np.mean(scores['test_score']))

0.855300214703487


In [16]:
# 위 코드와 똑같은 코드
scores = cross_validate(dt, train_input , train_target , cv= StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [18]:
# 10-폴드 교차 검증
splitter = StratifiedKFold(n_splits=10 , shuffle=True , random_state=42)
scores = cross_validate(dt, train_input , train_target , cv= splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


# 하이퍼파라미터튜닝 (편하게 임포트 이용)

> 사용자가 지정해야만하는 파라미터

> 검증 세트의 점수나 교차 검증을 통해 매개변수를 조금씩 바꿔봄

> 그리드 서치 : 훈련이 끝나면 25개의 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련.

In [20]:
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004 , 0.0005] }

In [21]:
# 그리드 서치 객체 생성
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params , n_jobs=-1)

> 결정 트리 클래스의 객체를 생성하자마자 바로 전달

In [22]:
gs.fit(train_input ,train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [23]:
dt = gs.best_estimator_
print(dt.score(train_input ,train_target))

0.9615162593804117


> 최적의 매개변수는 best_params_ 속성에 저장

In [24]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [25]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [26]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


## 과정 정리

1. 먼저 탐색할 매개변수 지정
2. 그다음 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾음.(이 조합은 그리드 서치 객체에 저장됨)
3. 그리드 서치는 최상의 매개변수에서 (교차 검증에 사용한 훈련 세트가 아니라) 전체 훈련 세트를 사용해 최종 모델을 훈련 (이 모델도 그리드 서치 객체에 저장됨)

In [27]:
# 더 복잡한 매개변수 조합 탐색
params = {'min_impurity_decrease' : np.arange(0.0001 , 0.001, 0.0001),
         'max_depth': range(5,20,1),
         'min_samples_split' : range(2,100,10)}

### 함수 설명

np.arange(0.0001 , 0.001, 0.0001) : 첫번째 매개변수 값에서 시작하여 두번째 매개변수에 도달할 때까지 세번째 매개변수를 계속 더한 배열을 만듬

range(x,y,z) : 정수만 사용 가능하며 .x에서 y까지 z씩 증가 하면서 값을 생성

In [28]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input , train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [29]:
# 최상의 매개변수 조합 확인
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [30]:
# 최상의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


# 랜덤 서치 (임포트)
> 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달

In [35]:
# 0에서 10 사이의 범위를 갖는 randint 객체를 만들고 10개의 숫자를 샘플링
rgen = randint(0 ,10)
rgen.rvs(10)

array([3, 7, 6, 4, 5, 1, 3, 5, 5, 3])

In [36]:
# 1000개 샘플링
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([109,  86,  96,  96, 104, 115,  97, 103, 101,  93]))

In [37]:
# 0~1 의 10개 실수 추출
ugen = uniform(0,1)
ugen.rvs(10)

array([0.46214935, 0.47391698, 0.08113901, 0.0342477 , 0.411983  ,
       0.79979556, 0.32456457, 0.33591199, 0.89279569, 0.63834879])

In [43]:
# 탐색할 딕셔너리 생성
params = {'min_impurity_decrease' : uniform(0.0001 , 0.001),
         'max_depth': randint(20,50),
         'min_samples_split' : randint(2,25),
         'min_samples_leaf' : randint(1,25)}

In [44]:
# 랜덤 서치 클래스 임포트
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100 , n_jobs=-1 , random_state=42)
gs.fit(train_input , train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ffa8507b820>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ffa85e15fd0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ffa85084100>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ffa85078fa0>},
                   random_state=42)

In [46]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [47]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [49]:
dt = gs.best_estimator_
print(dt.score(test_input ,test_target))

0.86


> 수동으로 하는거보다 그리드 서치나 랜덤 서치가 편하다.